In [1]:
%matplotlib inline
import os
import pandas as pd
from subprocess import PIPE, Popen
from glob import glob
from datetime import timedelta

# Corporacion Favorita

In [2]:
#A sales forcasting problem for the Ecuadorian Grocery chain Corporacion Favorita
#https://www.kaggle.com/c/favorita-grocery-sales-forecasting

## Let's take a look at the competition data

In [3]:
DATA_DIR = '../data/'
os.listdir(DATA_DIR)

['median_ma8.csv.gz',
 'oil.csv',
 'test.csv',
 'sample_submission.csv',
 'stores.csv',
 'items.csv',
 'train.csv',
 'holidays_events.csv',
 'transactions.csv']

## Inspect files by commandline rather than loading with pandas

In [4]:
for fl in sorted(glob(DATA_DIR + '*.csv')):
    p = Popen(['head', DATA_DIR + '{}'.format(fl)], stdin=PIPE, stdout=PIPE, stderr=PIPE)
    output, err = p.communicate()
    print('#'*20 + '  Head of file: {}'.format(fl))
    output = output.decode('ascii')
    for ll in output.split('\n'):
        print(ll)

####################  Head of file: ../data/holidays_events.csv
date,type,locale,locale_name,description,transferred
2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
2012-06-23,Holiday,Local,Guaranda,Cantonizacion de Guaranda,False
2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False

####################  Head of file: ../data/items.csv
item_nbr,family,class,perishable
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1
105574,GROCERY I,1045,0
105575,GROCERY I,1045,0
105576,GROCERY I,1045,0
105577,GROCERY I,1

![title](schema.jpg)

In [5]:
# Above from: https://www.kaggle.com/jeru666/all-csv-files-a-glance

In [6]:
# Under the hypotheses that grocery sales volumes are impacted by Ecuadorian holidays and oil prices,
# files with this information were included. 
# We are also given information about an Earthquake and that employees are typically paid on the 15th
# and the last day of the month.
# This tabular dataset of contextualized data fields is amenable to exploring feature engineering 
# and common-sense heuristics

In [7]:
# The sample_submission file shows that for each store id, we want to predict unit_sales
# Note the evaluation metric penalizes error in perishable items more severely
# https://www.kaggle.com/c/favorita-grocery-sales-forecasting#evaluation

## Load train.csv

In [8]:
train = pd.read_csv(DATA_DIR + 'train.csv')

/home/funk/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
train.head(20)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN
5,5,2013-01-01,25,108786,3.0,NaN
6,6,2013-01-01,25,108797,1.0,NaN
7,7,2013-01-01,25,108952,1.0,NaN
8,8,2013-01-01,25,111397,13.0,NaN
9,9,2013-01-01,25,114790,3.0,NaN


In [10]:
if train.date.is_monotonic_increasing:
    print('2017 instances begin at index {}'.format(len(train.query("date < '2017-01-01'"))))

2017 instances begin at index 101688779


In [11]:
unit_sales_stats = dict(train['unit_sales'].describe())

In [12]:
unit_sales_stats

{'25%': 2.0,
 '50%': 4.0,
 '75%': 9.0,
 'count': 125497040.0,
 'max': 89440.0,
 'mean': 8.5548652684376716,
 'min': -15372.0,
 'std': 23.605151805092405}

In [13]:
# Above, we find that unit sales vary across several orders of magnitude suggesting the log transform

## Simple baseline using the median of moving averages over several time scales

In [14]:
# based on: https://www.kaggle.com/paulorzp/log-moving-averages-forecasting-lb-0-546/

In [15]:
dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8'}       # use less memory, csv is 4.7 GB
converters = {'unit_sales':lambda u: float(u) if float(u)>0 else 0}   # coerce negatives to 0
data_pre_2017 = range(1, 101688779)                                   # keep header, drop data prior to 2017   
train = pd.read_csv(DATA_DIR + 'train.csv', usecols=[1,2,3,4], 
                    dtype=dtypes, parse_dates=['date'],
                    converters=converters,
                    skiprows=data_pre_2017
                   )
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p)         # Apply log1p for RMSLE, NOTE: IGNORES WEIGHTS
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date', 'store_nbr', 'item_nbr']
    )   
)

del u_dates, u_stores, u_items                                         # release memory, kaggle kernel

train.loc[:, 'unit_sales'].fillna(0, inplace=True)                     # replace NaNs
train.reset_index(inplace=True)                                        # reset index and restoring unique columns  
lastdate = train.iloc[train.shape[0]-1].date

test = pd.read_csv(DATA_DIR + 'test.csv', usecols=[0,2,3], dtype=dtypes)
test = test.set_index(['item_nbr', 'store_nbr'])
ltest = test.shape[0]

MA_windows = [1,3,7,14,28,56,112,224]
#Moving Averages
for i in MA_windows:
    val='MA'+str(i)
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmp1 = tmp.groupby(['item_nbr', 'store_nbr'])['unit_sales'].mean().to_frame(val)
    test = test.join(tmp1, how='left')

#Median of MAs
test['unit_sales']=test.iloc[:,1:].median(axis=1)
test.loc[:, 'unit_sales'].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1)             # restoring unit values 
test[['id','unit_sales']].to_csv(DATA_DIR + 'median_ma8.csv.gz', index=False, float_format='%.3f', 
                                 compression='gzip')

In [16]:
# Submission scores approximately 0.546 on the public leaderboard, 2 mo. since launch scoring 50th percentile!
# Surprisingly good performance considering this model uses simple statistics

### This model assumes:
*  Recent unit_sales are sufficient
*  is_perishable weighting is negligible
*  moving avg over various time scales suffices to make prediction
*  distributional skew makes the more robust median statistic appropriate for aggregation
*  other data sources are less informative
*  information loss in coercing negatives to 0 of minor impact

### Further Directions:
*  Introduce weighting for the RMSLWE
*  Explore relationship between oil prices, day-of-month, is_holiday, onpromotion and unit_sales
*  Cluster stores, assumes greater statistical efficiency
*  Explore tree-based models
*  Use more training data
*  Generalize to a function of dataframe, filters, aggregators